# Version 2.5: 파인다이닝/스시오마카세 요약문 생성 (배치 모드)

이 노트북은 **리뷰 데이터를 사용하지 않고**, 외부 공식 정보만으로 파인다이닝/스시오마카세 매장의 요약문을 **일괄 생성**합니다.

## 📌 핵심 특징

### 데이터 소스 (4가지 소스 타입 기반)
- 📝 **restaurant_review** (매장 특징 및 평가)
  - 미쉐린 가이드, 블루리본 평가
  - 전문가 리뷰 및 수상 이력
  
- 🍽️ **signature_menu** (시그니처 메뉴 설명)
  - 대표 메뉴 및 코스 구성
  - 특별한 조리 기법이나 식재료
  
- 🏛️ **atmosphere** (분위기)
  - 인테리어 및 공간 디자인
  - 다이닝 경험 및 서비스 스타일
  
- 💡 **visit_tips** (방문 팁)
  - 예약 방법 및 드레스 코드
  - 추천 방문 시간대

### 프롬프트 아키텍처
- **시스템/유저 프롬프트 분리 패턴** 사용
- **Few-shot Learning**: 10개의 고품질 예시로 학습
- **JSON 출력 강제**: 구조화된 결과 보장

### 출력 형식
```json
{
  "shop_seq": 1,
  "shop_name": "모수",
  "title": "고요한 정원 미학을 담아낸 사계절 가이세키 플레이팅",
  "summaries": [
    "셰프 철학/브랜드 정체성 문장 (40-60자)",
    "코스 구성/시그니처 특징 문장 (40-60자)",
    "공간/분위기 문장 (40-60자)"
  ]
}
```

## 🚀 사용 워크플로우

```
1. 섹션 1-3: 환경 설정 (최초 1회)
2. 섹션 4: SHOPS 리스트에 매장 정보 입력
3. 섹션 5: 건너뛰기 (섹션 4에서 정보 입력 완료)
4. 섹션 6: 프롬프트 실행 → 여러 매장 일괄 생성
5. 섹션 7: 결과 검증 (선택)
6. 섹션 8: JSON + CSV 파일 저장 (선택)
```

## ⚠️ 섹션 1-3: 최초 1회만 실행 (수정 금지)

In [ ]:
# 섹션 1: Package 설치
%pip install google-cloud-aiplatform google-genai python-dotenv

In [ ]:
# 섹션 2: 라이브러리 Import
import os
import json
import pandas as pd
import vertexai
from google import genai
from google.genai import types
from dotenv import load_dotenv

print("✅ 라이브러리 Import 완료")

In [ ]:
# 섹션 3: Vertex AI 초기화
# 환경 변수 로드
load_dotenv()

# GCP 프로젝트 설정
PROJECT_ID = "wad-dw"  # 실제 GCP 프로젝트 ID (프로젝트 이름 아님!)
LOCATION = "us-central1"

# Vertex AI 초기화
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

## ✅ 섹션 4: 매장 정보 입력 (수정 가능)

### 📋 입력 방법

1. **SHOPS 리스트**: 매장 기본 정보 (shop_seq, shop_name)
2. **COLLECTED_INFO 딕셔너리**: 매장별 수집된 정보를 매장명을 키로 입력

In [ ]:
# ========================================
# 매장 정보 입력
# ========================================

# 1️⃣ 매장 기본 정보 (shop_seq, shop_name)
SHOPS = [
    (1, "모수"),
    (2, "밍글스"),
    (3, "권숙수"),
]

# 2️⃣ 매장별 수집된 정보 (매장명을 키로 사용)
COLLECTED_INFO = {
    "모수": "",
    "밍글스": "",
    "권숙수": "",
}

# 3️⃣ SHOP_LIST 자동 생성 (shop_seq, shop_name, collected_info 매핑)
SHOP_LIST = [
    {
        "shop_seq": seq,
        "shop_name": name,
        "collected_info": COLLECTED_INFO.get(name, "[정보 없음 - COLLECTED_INFO에 추가 필요]")
    }
    for seq, name in SHOPS
]

# 출력
print(f"📍 다중 매장 배치 모드")
print(f"   - 총 매장 수: {len(SHOP_LIST)}개\n")
print("=" * 80)
for idx, shop in enumerate(SHOP_LIST, 1):
    info_length = len(shop['collected_info'].strip())
    status = "✅" if info_length > 100 else "⚠️"
    print(f"{status} [{idx}] {shop['shop_name']} (seq: {shop['shop_seq']}) - {info_length:,}자")
print("=" * 80)

if any(len(shop['collected_info'].strip()) < 100 for shop in SHOP_LIST):
    print("\n⚠️ 일부 매장의 정보가 부족합니다. COLLECTED_INFO를 확인하세요.")

## ✅ 섹션 5: 소스 타입별 정보 구조 안내

### 📂 파인다이닝/스시오마카세 소스 타입 (4가지)

섹션 4의 `COLLECTED_INFO`에 입력할 정보는 아래 4가지 소스 타입을 포함해야 합니다:

#### 1️⃣ restaurant_review (매장 특징 및 평가)
- 미쉐린 가이드, 블루리본 평가
- 전문가 리뷰 및 수상 이력
- 매장의 전반적인 명성과 평가

#### 2️⃣ signature_menu (시그니처 메뉴 설명)
- 대표 메뉴 및 시그니처 요리
- 코스 구성 및 메뉴 특징
- 특별한 조리 기법이나 식재료

#### 3️⃣ atmosphere (분위기)
- 인테리어 및 공간 디자인
- 다이닝 경험 및 서비스 스타일
- 좌석 배치 및 공간 분위기

#### 4️⃣ visit_tips (방문 팁)
- 예약 방법 및 난이도
- 드레스 코드 및 방문 시 유의사항
- 추천 방문 시간대 또는 특별 이벤트

### 💡 정보 입력 예시

```python
COLLECTED_INFO = {
    "모수": """
    ## RESTAURANT_REVIEW
    - 미쉐린 1스타 레스토랑
    - 블루리본 수상 이력
    - 한국 모던 한식의 선두주자로 평가받음
    
    ## SIGNATURE_MENU
    - 화덕에서 구운 우엉칩
    - 제철 식재료를 활용한 10가지 코스
    - 발효 소스를 활용한 메인 요리
    
    ## ATMOSPHERE
    - 모던하고 미니멀한 인테리어
    - 오픈 키친으로 조리 과정 관람 가능
    - 격식 있으면서도 편안한 다이닝 공간
    
    ## VISIT_TIPS
    - 예약 필수 (최소 2주 전)
    - 스마트 캐주얼 드레스 코드
    - 런치 코스가 디너보다 접근하기 쉬움
    """
}
```

⚠️ **주의**: 이 섹션은 정보 구조를 안내하는 용도입니다. 실제 정보는 섹션 4의 `COLLECTED_INFO`에 입력하세요.

In [ ]:
# ========================================
# 정보 입력 확인
# ========================================

print("📥 매장별 정보 입력 현황\n")
print("=" * 80)

for idx, shop in enumerate(SHOP_LIST, 1):
    shop_name = shop['shop_name']
    collected_info = shop['collected_info']
    info_length = len(collected_info.strip())
    
    # 상태 표시
    if info_length < 50:
        status = "❌ 정보 없음"
    elif info_length < 200:
        status = "⚠️ 정보 부족"
    else:
        status = "✅ 정보 충분"
    
    print(f"[{idx}] {shop_name}")
    print(f"    상태: {status} ({info_length:,}자)")
    print(f"    권장: 각 소스 타입당 100~500자 (총 400~2000자)")
    print()

print("=" * 80)
print("\n⚠️ 정보가 부족한 매장은 섹션 4의 COLLECTED_INFO를 수정하세요.")
print("➡️ 정보 입력이 완료되었으면 섹션 6을 실행하세요.")

## ✅ 섹션 6: 프롬프트 작성 및 요약문 생성 (수정 가능)

### 프롬프트 구조 설계

이 노트북은 **시스템/유저 프롬프트 분리 패턴**을 사용합니다:

#### 📌 시스템 프롬프트 (System Prompt)
- **역할**: AI의 정체성, 규칙, 출력 형식 정의
- **특징**: 고정된 내용 (매장마다 동일)
- **내용**: 작성 원칙, 구조 규칙, 출력 형식

#### 📌 유저 프롬프트 (User Prompt)
- **역할**: 구체적인 작업 요청 및 데이터 제공
- **특징**: 매장마다 변경되는 내용
- **내용**: 매장 정보, 수집된 정보, Few-shot 예시

#### 🔗 프롬프트 결합
Google Gemini는 시스템/유저를 API 레벨에서 구분하지 않으므로, 전송 시 하나의 문자열로 결합합니다.

### 프롬프트 작성 규칙
- ❌ **f-string 사용 금지** (변수가 아직 정의되지 않아 에러)
- ✅ **일반 문자열 + `.format()` 사용**
- ✅ **JSON 예시에서 `{{` `}}` 사용** (이스케이프)

In [ ]:
# ========================================
# 프롬프트 정의
# ========================================

# -----------------------------------------
# [시스템 프롬프트] AI의 정체성과 규칙
# -----------------------------------------
system_prompt = """
당신은 미쉐린 가이드와 블루리번 수준의 파인다이닝 레스토랑 전문 작가입니다.
외부 공식 정보(미쉐린 가이드, 블루리번, 셰프 인터뷰 등)를 바탕으로 매장의 정체성을 압축적으로 표현하는 요약문을 작성합니다.

## 작성 원칙
1. **사실 기반**: 공식 출처의 정보만 사용 (추측 금지)
2. **구조화**: 제목 1개 + 요약 문장 3개로 구성
3. **간결성**: 각 문장은 약 40-60자, 핵심만 전달
4. **전문성**: 미식 용어를 정확하게 사용 (예: 에도마에, 가이세키, 누룩, 숙성)
5. **객관성**: 과장된 표현 금지, 감성적 수식어 최소화
6. **독창성**: 예시를 그대로 복사하지 말고, 톤앤매너만 참고하여 해당 매장만의 고유한 특징을 작성

## 소스 타입별 정보 활용 가이드

수집된 정보는 4가지 소스 타입으로 구성됩니다:

1. **RESTAURANT_REVIEW** (매장 특징 및 평가)
   - 미쉐린/블루리번 평가, 수상 이력
   - 매장의 명성과 전문가 평가
   → 주로 **1번 문장**(셰프 철학/브랜드 정체성)에 활용

2. **SIGNATURE_MENU** (시그니처 메뉴 설명)
   - 대표 메뉴, 코스 구성, 조리 기법
   - 특별한 식재료나 메뉴 특징
   → 주로 **2번 문장**(코스 구성/시그니처 메뉴)과 **제목**에 활용

3. **ATMOSPHERE** (분위기)
   - 인테리어, 공간 디자인, 다이닝 경험
   - 서비스 스타일, 좌석 배치
   → 주로 **3번 문장**(공간 특징/분위기)에 활용

4. **VISIT_TIPS** (방문 팁)
   - 예약 방법, 드레스 코드, 추천 시간대
   - 특별 이벤트나 유의사항
   → 필요시 **3번 문장**에 보조 정보로 활용

## 요약 문장 구조
- **1번 문장**: 셰프 철학 / 브랜드 정체성 / 조리 철학
  - RESTAURANT_REVIEW + SIGNATURE_MENU 정보 활용
- **2번 문장**: 코스 구성 / 시그니처 메뉴 / 조리 기법
  - SIGNATURE_MENU 정보 중심 활용
- **3번 문장**: 공간 특징 / 분위기 / 미식 경험
  - ATMOSPHERE + VISIT_TIPS 정보 활용

## 제목 작성 규칙
- 형식: "[공간 특징] 속, [시그니처 요소]를 [동사]하는 [장르]"
- 예시: "고요한 정원 미학을 담아낸 사계절 가이세키 플레이팅"
- 길이: 15-30자
- SIGNATURE_MENU와 ATMOSPHERE 정보 활용

## 출력 형식
반드시 유효한 JSON 형식으로만 출력하세요. 다른 설명이나 주석은 포함하지 마세요.
{{
  "shop_seq": 숫자,
  "shop_name": "매장명",
  "title": "제목",
  "summaries": ["문장1", "문장2", "문장3"]
}}
"""

# -----------------------------------------
# [유저 프롬프트 템플릿] 구체적 작업 내용
# -----------------------------------------
user_prompt_template = """
[매장 정보]
매장명: {shop_name}
Shop Seq: {shop_seq}

[수집된 정보 - 소스 타입별]
{collected_info}

[참고 예시 - 아래 예시들의 톤앤매너와 문체만 참고하세요. 내용은 절대 복사하지 마세요.]

**중요**: 아래 예시는 작성 스타일을 보여주기 위한 참고용입니다. 예시의 내용을 그대로 사용하지 말고, 
수집된 정보를 바탕으로 해당 매장만의 고유한 특징을 작성하세요.

---

**예시 1: 모수 서울(Mosu Seoul)**
{{
  "title": "식재료에 대한 존중이 돋보이는 이노베이티브 퀴진",
  "summaries": [
    "모수 서울은 한국 미식의 위상을 높였다는 평가를 받는 이노베이티브 한식 다이닝입니다. 현지 식재료에 대한 존중을 바탕으로 창의적인 한식을 선보입니다.",
    "시그니처 메뉴인 엉걸불 도토리 국수는 진한 버터와 트러플의 조화가 깊은 풍미를 냅니다. 두유 껍질로 만든 전복 타코는 숙성 전복과 씨겨자의 조화가 인상적입니다.",
    "화이트 톤과 나무 소재를 활용한 공간은 편안하면서도 세련된 분위기를 자아냅니다. 로제 샴페인, 산도 있는 화이트 와인, 이탈리아 레드 와인 등을 코스의 흐름에 맞춘 페어링 메뉴도 인기가 높습니다."
  ]
}}

**예시 2: 밍글스**
{{
  "title": "국내 유일 3스타의 영예를 안은 모던 한식 다이닝",
  "summaries": [
    "밍글스는 전통 장과 서양 조리 기법을 접목해 한식의 새로운 경계를 탐구하는 미쉐린 3스타 다이닝입니다.",
    "대표 디저트 '장트리오'는 장의 고소함과 쌀의 단맛이 조화를 이루어 가장 많이 언급됩니다. 숯불로 구운 한우 스테이크는 육즙이 풍부하고 익힘 정도가 좋아 메인으로 인기가 높습니다.",
    "큰 창을 통해 정원을 조망할 수 있는 공간은 우아하고 정제된 분위기입니다. 청명주와 청감주 등 한국 전통주를 활용한 페어링은 한식의 깊은 맛과 장의 풍미에 새로운 시너지를 만들어냅니다."
  ]
}}

**예시 3: 정식당**
{{
  "title": "뉴 코리안 장르를 선도하는 한식 파인 다이닝",
  "summaries": [
    "정식당은 '뉴 코리안'이라는 장르를 개척하며 전통 한식을 현대적인 감각으로 재해석한 미쉐린 2스타 다이닝입니다.",
    "시그니처인 장독대를 형상화한 디저트와 더불어 전복과 트러플 소스를 곁들인 '맛있는 김밥'이 꾸준히 좋은 평가를 받습니다.",
    "고급스럽고 격식 있는 현대적 공간은 창을 통해 도시의 전경을 조망합니다. 부르고뉴 와인이나 고급 샴페인 등을 활용한 페어링은 요리와 와인 간의 균형을 섬세하게 맞춥니다."
  ]
}}

**예시 4: 알라프리마**
{{
  "title": "즉흥 속에서 맛의 균형을 완성하는 파인 다이닝",
  "summaries": [
    "알라프리마는 매일 최상의 제철 재료를 바탕으로 즉흥적인 코스를 구성하는 미쉐린 2스타 다이닝입니다. 불필요한 장식을 배제하고 재료 본연의 맛에 집중합니다.",
    "시그니처 메뉴인 '40가지 야채 샐러드'는 에멘탈 치즈 유청 소스와 따뜻한 드레싱의 조화로 인기를 많습니다. 바삭하게 튀긴 옥돔 껍질과 뼈 육수의 조합은 진한 감칠맛을 냅니다.",
    "간결한 미니멀리즘 인테리어의 공간입니다. 와인 페어링은 슈냉 블랑 등의 화이트 와인을 중심으로 구성되며, 즉흥적인 제철 코스에 맞춰 정교한 밸런스를 유지합니다."
  ]
}}

**예시 5: 권숙수**
{{
  "title": "한국의 고급 독상 문화를 현대적으로 재해석한 다이닝",
  "summaries": [
    "전통적인 독상 문화를 현대적 코스로 재구성한 미쉐린 2스타 다이닝입니다. 한국 전통과 발효의 미학을 깊이 있게 반영합니다.",
    "대표 메뉴인 된장으로 구워 냄새를 잡은 양갈비는 고기의 풍미가 농후하여 메인으로 선호됩니다. 고소하고 향이 깊은 트러플 콩국수와 민어구이 등 제철을 반영한 요리들이 깊은 인상을 남깁니다.",
    "전면 유리창을 통해 외부 조경을 감상할 수 있으며, 프라이빗 룸이 마련된 품격 있는 공간입니다. 석탄주, 자주 등을 활용한 전통주 페어링은 한식 코스의 격조를 한단계 상승시킵니다."
  ]
}}

**예시 6: 스시마츠모토**
{{
  "title": "세 가지 온도 샤리로 네타의 맛을 극대화하는 스시야",
  "summaries": [
    "저온, 중온, 고온의 세 가지 샤리 온도를 재료에 따라 구분하여 사용하는 것이 큰 특징인 미쉐린 1스타 스시야입니다. 숙련된 기술로 재료의 맛을 일관되게 구현하는 것으로 유명합니다.",
    "복찜과 게우소스를 남은 샤리에 비벼 먹는 조합이 꾸준히 좋은 평가를 받습니다. 기름진 니싱(청어)과 풍부한 맛이 좋은 주도로(참치 중뱃살) 초밥이 입안에서 사르르 녹는 질감을 보여줍니다.",
    "정갈한 우드 카운터 공간에서 스시의 맛과 향에 집중하는 경험을 제공합니다. 하우스 사케가 과실향이 뚜렷하여 츠마미와 니기리 모두에 잘 어울린다는 평가입니다."
  ]
}}

**예시 7: 하네**
{{
  "title": "한국 식재료를 활용하는 섬세한 스시 오마카세",
  "summaries": [
    "한국의 제철 식재료를 활용하여 섬세하고 정교한 스시를 선보이는 것이 큰 특징인 미쉐린 1스타 스시 오마카세입니다. 독창적인 츠마미 구성으로 주목 받습니다.",
    "츠마미는 불향이 은은하게 올라오는 갯장어와 매실소스의 조합이 호평을 받습니다. 간이 절제된 소스의 대게살 요리와 풍미가 좋은 전어 등 등 푸른 생선 조리에 강점을 보입니다.",
    "전통적인 소재들을 현대적으로 표현한 세련되고 고급스러운 공간입니다. 하네야 카스미자케와 같은 과실향이 풍부한 사케가 비가열 요리와 잘 어울린다는 평가가 많습니다."
  ]
}}

**예시 8: 소수헌**
{{
  "title": "35년 장인 정신을 담아내는 스시야",
  "summaries": [
    "소수헌은 35년간 축적된 스시 장인 정신을 기반으로, 샤리와 네타의 볼륨감이 살아있는 니기리를 선보이는 것이 큰 특징인 미쉐린 1스타 스시야입니다.",
    "코스 중 완벽한 숙성과 온도감을 자랑하는 참치 뱃살(오도로)과 깊은 풍미의 보탄에비(단새우) 초밥이 호평을 받습니다. 바삭하고 맛있는 김을 사용한 간뾰 마끼 역시 만족도가 높습니다.",
    "한옥을 리모델링한 고즈넉한 공간은 도심 속 스카이라인을 배경으로 깊이 있는 미식 경험을 제공합니다. 은은한 단맛과 향이 풍부한 고급 사케를 곁들이는 것을 추천합니다."
  ]
}}

**예시 9: 스시사이토 쥬욘**
{{
  "title": "일본 명가의 기법을 계승한 스시야",
  "summaries": [
    "스시사이토 쥬욘은 일본 미쉐린 3스타 '스시 사이토'의 철학을 이어받아 엄격한 수준의 기술로 조리 방식을 구현하는 곳입니다.",
    "샤리의 온도와 초의 비율을 완벽하게 제어해 네타와의 조화를 완성합니다. 산미가 깔끔한 참치 붉은살 스시와 익힘이 절묘한 금태 구이가 대표 메뉴입니다.",
    "깔끔한 우드톤 카운터가 정갈하게 배치되어 있어 오롯이 미식 경험에 집중할 수 있는 분위기입니다. 식사와 맛의 균형을 해치지 않는 깔끔한 신슈카레이 또는 돔 페리뇽과 같은 고급 샴페인 페어링을 추천합니다."
  ]
}}

**예시 10: 이루히 니와**
{{
  "title": "여의도의 축복 스시 정통 오마카세",
  "summaries": [
    "아루히 니와는 스강신청이라는 신조어가 탄생한 가성비 오마카세로 셰프의 유쾌한 감각과 구성력이 돋보이는 곳입니다.",
    "바삭한 튀김류가 호평을 받으며, 청어와 표고버섯을 말아 만든 이소베마끼는 향이 독특해 인기가 많습니다. 매생이와 트러플 오일을 곁들인 차완무시는 부드럽게 즐길 수 있습니다.",
    "일반적인 오마카세보다 밝고 즐거운 분위기를 제공합니다. 주류 주문은 필수이며 유자 사케처럼 가볍고 달콤한 주류가 초반 식중주로 잘 어울린다는 후기가 많습니다."
  ]
}}

---

**다시 한 번 강조**: 위 예시는 작성 스타일(톤, 문체, 구조)을 참고하기 위한 것입니다.
예시의 구체적인 내용(메뉴명, 셰프 이름, 철학 등)을 그대로 사용하지 말고,
[수집된 정보]를 바탕으로 [{shop_name}] 매장만의 고유한 특징을 작성하세요.
"""

print("✅ 프롬프트 템플릿 작성 완료")
print("   - 시스템 프롬프트: AI의 정체성과 규칙 + 소스 타입별 매핑 가이드")
print("   - 유저 프롬프트: 구체적 작업 내용")
print("   - Few-shot 예시: 10개 (톤앤매너 참고용)")

In [ ]:
# ========================================
# 모델 설정 및 LLM 호출
# ========================================

# -----------------------------------------
# 모델 파라미터 설정
# -----------------------------------------
MODEL_NAME = "gemini-2.5-pro"  # 높은 품질
TEMPERATURE = 0.5  # 낮은 창의성, 일관성 우선
MAX_OUTPUT_TOKENS = 4096  # JSON 응답을 위한 충분한 토큰

generation_config = types.GenerateContentConfig(
    temperature=TEMPERATURE,
    max_output_tokens=MAX_OUTPUT_TOKENS,
    response_mime_type="application/json"  # JSON 출력 강제
)

print(f"🤖 모델: {MODEL_NAME}")
print(f"   - Temperature: {TEMPERATURE}")
print(f"   - Max Output Tokens: {MAX_OUTPUT_TOKENS:,}\n")

# -----------------------------------------
# 요약문 생성 함수
# -----------------------------------------
def generate_summary(shop_seq, shop_name, collected_info):
    """단일 매장에 대한 요약문 생성"""
    
    # 유저 프롬프트 생성
    user_prompt = user_prompt_template.format(
        shop_seq=shop_seq,
        shop_name=shop_name,
        collected_info=collected_info
    )
    
    # 프롬프트 결합
    combined_prompt = f"{system_prompt}\n\n{user_prompt}"
    
    try:
        # LLM 호출
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=combined_prompt,
            config=generation_config
        )
        
        # JSON 파싱
        result_text = response.text
        result_json = json.loads(result_text)
        
        # 토큰 사용량 추출
        usage_metadata = response.usage_metadata if hasattr(response, 'usage_metadata') else None
        token_info = {}
        if usage_metadata:
            token_info = {
                "input_tokens": usage_metadata.prompt_token_count,
                "output_tokens": usage_metadata.candidates_token_count,
                "total_tokens": usage_metadata.total_token_count
            }
        
        return {
            "status": "success",
            "data": result_json,
            "tokens": token_info
        }
        
    except json.JSONDecodeError as e:
        return {
            "status": "error",
            "error_type": "JSONDecodeError",
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {}
        }
        
    except Exception as e:
        return {
            "status": "error",
            "error_type": type(e).__name__,
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {}
        }

# -----------------------------------------
# 다중 매장 배치 처리
# -----------------------------------------
all_results = []
total_input_tokens = 0
total_output_tokens = 0
total_tokens = 0

print(f"📝 {len(SHOP_LIST)}개 매장 요약문 생성 시작\n")
print("=" * 80)

for idx, shop in enumerate(SHOP_LIST, 1):
    shop_seq = shop["shop_seq"]
    shop_name = shop["shop_name"]
    collected_info = shop["collected_info"]
    
    print(f"\n[{idx}/{len(SHOP_LIST)}] {shop_name} 처리 중...")
    
    result = generate_summary(shop_seq, shop_name, collected_info)
    
    if result["status"] == "success":
        result_json = result["data"]
        all_results.append(result_json)
        
        # 토큰 사용량 출력 및 누적
        tokens = result.get("tokens", {})
        if tokens:
            input_tokens = tokens.get('input_tokens', 0)
            output_tokens = tokens.get('output_tokens', 0)
            total_tokens_shop = tokens.get('total_tokens', 0)
            
            total_input_tokens += input_tokens
            total_output_tokens += output_tokens
            total_tokens += total_tokens_shop
            
            print(f"   ✅ 성공 | Input: {input_tokens:,} | Output: {output_tokens:,} | Total: {total_tokens_shop:,} tokens")
        else:
            print(f"   ✅ 성공")
    else:
        print(f"   ❌ 실패: {result['error_type']} - {result['message']}")
        # 실패한 경우에도 기록
        all_results.append({
            "shop_seq": shop_seq,
            "shop_name": shop_name,
            "status": "error",
            "error": result['message']
        })

print("\n" + "=" * 80)
print(f"\n✅ 전체 완료: {len(all_results)}개 매장 처리")

# 성공/실패 개수
success_count = sum(1 for r in all_results if r.get("status") != "error")
error_count = len(all_results) - success_count
print(f"   - 성공: {success_count}개")
if error_count > 0:
    print(f"   - 실패: {error_count}개")

# 총 토큰 사용량 출력
if total_tokens > 0:
    print(f"\n📊 총 토큰 사용량:")
    print(f"   - Total Input:  {total_input_tokens:,} tokens")
    print(f"   - Total Output: {total_output_tokens:,} tokens")
    print(f"   - Total:        {total_tokens:,} tokens")

# -----------------------------------------
# 전체 결과 상세 출력
# -----------------------------------------
print("\n" + "=" * 80)
print("📋 전체 결과 상세보기")
print("=" * 80)

for idx, result in enumerate(all_results, 1):
    print(f"\n{'─' * 80}")
    print(f"[{idx}] {result.get('shop_name')}")
    print(f"{'─' * 80}")
    
    if result.get("status") != "error":
        # 성공한 경우 - 전체 내용 출력
        print(f"\n📌 제목:")
        print(f"   {result.get('title')}")
        
        print(f"\n📝 요약문:")
        summaries = result.get('summaries', [])
        for i, summary in enumerate(summaries, 1):
            print(f"   {i}. {summary}")
        
        print(f"\n✅ 상태: 성공")
    else:
        # 실패한 경우
        print(f"\n❌ 상태: 에러")
        print(f"   오류 내용: {result.get('error', 'N/A')}")

print("\n" + "=" * 80)

## ✅ 섹션 7: 후처리 검증 (선택 사항)

### 검증 항목

이 섹션은 생성된 요약문의 품질을 **코드 기반**으로 검증합니다 (LLM 호출 없음):

#### 📋 기본 구조 검증
- 필수 키 존재 확인 (shop_seq, shop_name, title, summaries)
- 요약문 개수 (정확히 3개)
- 문장 길이 (40-60자 권장, 30자 미만/100자 초과 경고)
- 제목 길이 (15-30자 권장)

#### 🚫 품질 검증
- 예시 복사 의심 (예시와 90% 이상 유사)
- 과장된 표현 금지 (최고의, 완벽한, 최상의 등)
- 문장 간 중복 내용 (3개 문장이 너무 유사)
- 특수문자/이모지 체크

#### 💡 검증 결과
- ✅ 통과: 모든 규칙 준수
- ⚠️ 경고: 일부 권장사항 위반 (사용 가능하지만 검토 권장)
- ❌ 실패: 필수 규칙 위반 (수정 필요)

In [ ]:
# ========================================
# 강력한 후처리 검증 함수
# ========================================

import difflib
import re

# -----------------------------------------
# 예시 문장 리스트 (복사 의심 체크용)
# -----------------------------------------
EXAMPLE_SENTENCES = [
    "안성재 셰프는 현지 식재료에 대한 존중을 기반으로, 한국 재료에 셰프의 테크닉을 더한 새로운 형태의 모던 한식을 창조합니다.",
    "코스는 화덕에서 구워낸 우엉칩처럼 유년 시절의 경험을 재해석하며, 중식 요소를 접목하는 등 자유롭고 창의적인 조리 맥락을 보여줍니다.",
    "제철 재료의 온도와 향을 정교하게 다루는 조리 구조가 특징이며, 모던하고 깔끔한 다이닝 공간에서 전통과 현대적 균형을 구현합니다.",
    "강민구 셰프는 '서로 다른 것의 조화'를 핵심 철학으로 삼아, 전통 한식의 발효 기법과 장(醬)을 서양 요리 기법과 접목하여 맛의 세련된 균형을 구현합니다.",
    "코스는 제철 식재료를 중심으로 흑초 글레이징, 누룩 소스 등을 활용하며, 장을 활용한 크림브륄레 같은 메뉴로 한식 디저트의 미식적 확장을 보여줍니다.",
    "한국 장인의 수공예 식기와 목재 질감 인테리어가 조화를 이루는 우아한 공간에서 한식의 가치를 전달합니다.",
]

# -----------------------------------------
# 금지 키워드 리스트 (과장된 표현)
# -----------------------------------------
BANNED_KEYWORDS = [
    "최고의", "최상의", "완벽한", "최고급의", "세계 최고",
    "압도적인", "독보적인", "타의 추종을 불허하는",
    "놀라운", "경이로운", "환상적인", "굉장한"
]

def calculate_similarity(text1, text2):
    """두 문자열 간 유사도 계산 (0.0 ~ 1.0)"""
    return difflib.SequenceMatcher(None, text1, text2).ratio()

def check_example_similarity(sentence):
    """예시 문장과의 유사도 체크"""
    max_similarity = 0.0
    most_similar_example = None
    
    for example in EXAMPLE_SENTENCES:
        similarity = calculate_similarity(sentence, example)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_example = example
    
    return max_similarity, most_similar_example

def check_banned_keywords(text):
    """금지 키워드 체크"""
    found_keywords = []
    for keyword in BANNED_KEYWORDS:
        if keyword in text:
            found_keywords.append(keyword)
    return found_keywords

def check_special_chars(text):
    """특수문자/이모지 체크"""
    # 이모지 패턴
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # 이모티콘
        "\U0001F300-\U0001F5FF"  # 기호 & 픽토그램
        "\U0001F680-\U0001F6FF"  # 교통 & 지도
        "\U0001F1E0-\U0001F1FF"  # 국기
        "]+", 
        flags=re.UNICODE
    )
    
    emojis = emoji_pattern.findall(text)
    return emojis

def validate_summary(result, strict=False):
    """
    요약문 검증 함수
    
    Args:
        result: 검증할 결과 dict
        strict: True이면 경고도 실패로 간주
    
    Returns:
        dict: {
            "passed": bool,
            "errors": list,
            "warnings": list
        }
    """
    errors = []
    warnings = []
    
    # -----------------------------------------
    # 1. 필수 키 존재 확인
    # -----------------------------------------
    required_keys = ["shop_seq", "shop_name", "title", "summaries"]
    for key in required_keys:
        if key not in result:
            errors.append(f"❌ 필수 키 누락: {key}")
    
    if errors:  # 필수 키가 없으면 더 이상 검증 불가
        return {
            "passed": False,
            "errors": errors,
            "warnings": warnings
        }
    
    # -----------------------------------------
    # 2. 요약문 개수 확인
    # -----------------------------------------
    summaries = result.get("summaries", [])
    if len(summaries) != 3:
        errors.append(f"❌ 요약문 개수 오류: {len(summaries)}개 (3개 필요)")
        return {
            "passed": False,
            "errors": errors,
            "warnings": warnings
        }
    
    # -----------------------------------------
    # 3. 문장 길이 확인
    # -----------------------------------------
    for i, sentence in enumerate(summaries, 1):
        length = len(sentence)
        if length < 30:
            warnings.append(f"⚠️ {i}번 문장이 너무 짧음: {length}자 (권장 40-60자)")
        elif length > 100:
            warnings.append(f"⚠️ {i}번 문장이 너무 김: {length}자 (권장 40-60자)")
    
    # -----------------------------------------
    # 4. 제목 길이 확인
    # -----------------------------------------
    title = result.get("title", "")
    title_length = len(title)
    if title_length < 10:
        warnings.append(f"⚠️ 제목이 너무 짧음: {title_length}자 (권장 15-30자)")
    elif title_length > 40:
        warnings.append(f"⚠️ 제목이 너무 김: {title_length}자 (권장 15-30자)")
    
    # -----------------------------------------
    # 5. 예시 복사 의심 체크
    # -----------------------------------------
    for i, sentence in enumerate(summaries, 1):
        similarity, similar_example = check_example_similarity(sentence)
        if similarity >= 0.9:
            errors.append(f"❌ {i}번 문장이 예시를 복사한 것으로 의심됨 (유사도 {similarity:.1%})")
            errors.append(f"   원본: {similar_example[:50]}...")
        elif similarity >= 0.7:
            warnings.append(f"⚠️ {i}번 문장이 예시와 유사함 (유사도 {similarity:.1%})")
    
    # -----------------------------------------
    # 6. 금지 키워드 체크
    # -----------------------------------------
    all_text = title + " ".join(summaries)
    banned_found = check_banned_keywords(all_text)
    if banned_found:
        warnings.append(f"⚠️ 과장된 표현 발견: {', '.join(banned_found)}")
    
    # -----------------------------------------
    # 7. 특수문자/이모지 체크
    # -----------------------------------------
    emojis = check_special_chars(all_text)
    if emojis:
        warnings.append(f"⚠️ 이모지 발견: {', '.join(emojis)}")
    
    # -----------------------------------------
    # 8. 문장 간 중복 체크 (3개 문장이 너무 유사)
    # -----------------------------------------
    for i in range(len(summaries)):
        for j in range(i+1, len(summaries)):
            similarity = calculate_similarity(summaries[i], summaries[j])
            if similarity >= 0.7:
                warnings.append(f"⚠️ {i+1}번과 {j+1}번 문장이 너무 유사함 (유사도 {similarity:.1%})")
    
    # -----------------------------------------
    # 결과 판정
    # -----------------------------------------
    if strict:
        passed = len(errors) == 0 and len(warnings) == 0
    else:
        passed = len(errors) == 0
    
    return {
        "passed": passed,
        "errors": errors,
        "warnings": warnings
    }

# -----------------------------------------
# 전체 결과 검증 실행
# -----------------------------------------
if len(all_results) == 0:
    print("⚠️ 검증할 결과가 없습니다.")
    print("   먼저 섹션 6을 실행하여 요약문을 생성하세요.")
else:
    print("🔍 요약문 검증 시작\n")
    print("=" * 80)
    
    total_passed = 0
    total_warnings = 0
    total_failed = 0
    
    validation_results = []
    
    for idx, result in enumerate(all_results, 1):
        # 에러 상태인 결과는 스킵
        if result.get("status") == "error":
            print(f"\n[{idx}] {result.get('shop_name')}: 생성 실패로 검증 스킵")
            continue
        
        shop_name = result.get("shop_name", "Unknown")
        print(f"\n[{idx}] {shop_name}")
        print("─" * 80)
        
        # 검증 실행
        validation = validate_summary(result, strict=False)
        validation_results.append({
            "shop_name": shop_name,
            "validation": validation
        })
        
        errors = validation["errors"]
        warnings = validation["warnings"]
        passed = validation["passed"]
        
        if passed and len(warnings) == 0:
            print("✅ 검증 통과: 모든 규칙을 준수합니다")
            total_passed += 1
        elif passed:
            print(f"✅ 검증 통과 (경고 {len(warnings)}개)")
            total_passed += 1
            total_warnings += 1
            for warning in warnings:
                print(f"   {warning}")
        else:
            print(f"❌ 검증 실패 (오류 {len(errors)}개, 경고 {len(warnings)}개)")
            total_failed += 1
            for error in errors:
                print(f"   {error}")
            for warning in warnings:
                print(f"   {warning}")
    
    # -----------------------------------------
    # 전체 검증 요약
    # -----------------------------------------
    print("\n" + "=" * 80)
    print("📊 검증 요약")
    print("=" * 80)
    print(f"✅ 통과: {total_passed}개")
    if total_warnings > 0:
        print(f"⚠️ 경고 있음: {total_warnings}개")
    if total_failed > 0:
        print(f"❌ 실패: {total_failed}개")
    
    success_rate = (total_passed / len([r for r in all_results if r.get("status") != "error"])) * 100
    print(f"\n성공률: {success_rate:.1f}%")

## ✅ 섹션 8: 결과 저장 (선택 사항)

### 저장 형식

#### 단일 매장 모드
- JSON 파일 1개 (전체 구조 포함)
- 파일명: `summary_{매장명}_{타임스탬프}.json`

#### 다중 매장 모드
- JSON 파일 (전체 구조 포함)
- CSV 파일 (분석용 테이블 형식)
- 파일명: `summaries_batch_{타임스탬프}.json/csv`

In [ ]:
# ========================================
# 결과 저장
# ========================================

from datetime import datetime
import pandas as pd

if len(all_results) == 0:
    print("⚠️ 저장할 결과가 없습니다.")
else:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # JSON 파일
    json_filename = f"summaries_batch_{timestamp}.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)
    
    print(f"✅ JSON 저장 완료: {json_filename}")
    
    # CSV 파일
    csv_data = []
    for result in all_results:
        if result.get("status") != "error":
            csv_data.append({
                "shop_seq": result.get("shop_seq"),
                "shop_name": result.get("shop_name"),
                "title": result.get("title"),
                "summary_1": result.get("summaries", [])[0] if len(result.get("summaries", [])) > 0 else "",
                "summary_2": result.get("summaries", [])[1] if len(result.get("summaries", [])) > 1 else "",
                "summary_3": result.get("summaries", [])[2] if len(result.get("summaries", [])) > 2 else "",
                "status": "success"
            })
    
    df = pd.DataFrame(csv_data)
    csv_filename = f"summaries_batch_{timestamp}.csv"
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    
    print(f"✅ CSV 저장 완료: {csv_filename}")